In [1]:
!pip install pyvi 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.8 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
import wandb
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import (
    InformationRetrievalEvaluator,
    SequentialEvaluator,
)
from pyvi import ViTokenizer
from sentence_transformers import SentenceTransformerModelCardData
from sentence_transformers.util import cos_sim
from datasets import load_dataset, concatenate_datasets
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from transformers import  EarlyStoppingCallback

In [3]:
wandb.login(key = '8a5cbfdaa29778a896996cc679358b1d96cf66b0')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hdang1696 (hdang1696-no-work-experience). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## 1.Create & Prepare embedding dataset

In [4]:
df_legal=pd.read_csv("/kaggle/input/huy-embedding-data/final_train_1.csv")

In [5]:
df_legal.head(5)

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,context,question,answer,Unnamed: 0
0,Điều 30 . Tổ sát_hạch 1 . Tổ sát_hạch do Cục_t...,Tổ sát_hạch cấp giấy_phép lái tàu_hỏa có bao_n...,Điều 10 . Xác_định nội_dung đăng_ký lại khai_s...,NaN
1,Điều 11 . Tổ_chức đào_tạo nghề cho lao_động tr...,Văn_bản điều_chỉnh hoạt_động tổ_chức đào_tạo n...,"Điều 6 . Kê_khai , nộp phí 1 . Chậm nhất là ng...",NaN
2,"Điều 9 . Chế_độ phụ_cấp đặc_thù quốc_phòng , q...",Phó chỉ_huy_trưởng Ban chỉ_huy quân_sự xã được...,Điều 1 . Phạm_vi điều_chỉnh Thông_tư này hướng...,NaN
3,Điều 8 . Trách_nhiệm của người đứng đầu đoàn_t...,Thực_hiện dân_chủ trong cơ_sở giáo_dục công_lậ...,Điều 40 . Hiệu_lực thi_hành 1 . Nghị_định này ...,NaN
4,Điều 16 . Xử_phạt người điều_khiển xe ô_tô ( b...,Mức phạt nếu điều_khiển xe ô_tô lắp thêm đèn,Điều 23 . Nội_dung tin cảnh_báo sóng_thần 1 . ...,NaN


In [6]:
df_legal.shape

(71714, 4)

In [7]:
df_legal['question'] = df_legal['question'].apply(ViTokenizer.tokenize)
df_legal['context'] = df_legal['context'].apply(ViTokenizer.tokenize)

In [8]:
df_legal=df_legal[['question','context']]

In [9]:
df_legal = df_legal.rename(columns={'question': 'anchor', 'context': 'positive'})

In [10]:
df_legal["id"] = range(len(df_legal))

In [11]:

train_df_legal, temp_df_legal = train_test_split(df_legal, test_size=0.2, random_state=42)

# Chia temp thành tập test và val
test_df_legal, val_df_legal = train_test_split(temp_df_legal, test_size=0.5, random_state=42)

train_df_legal.to_json("/kaggle/working/train_dataset_legal.json", orient="records", lines=True)
test_df_legal.to_json("/kaggle/working/test_dataset_legal.json", orient="records", lines=True)
val_df_legal.to_json("/kaggle/working/val_dataset_legal.json", orient="records", lines=True)

## 2.Load Model Embedding Pre-train

In [12]:
model_id = "bkai-foundation-models/vietnamese-bi-encoder"  # Hugging Face model ID
matryoshka_dimensions = [768, 512, 256, 128, 64] # Important: large to small
 
# Load a model
model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

## 3.Load Dataset

In [13]:
test_dataset_legal = load_dataset("json", data_files="/kaggle/working/test_dataset_legal.json", split="train")
val_dataset_legal=load_dataset("json",data_files='/kaggle/working/val_dataset_legal.json',split='train')
train_dataset_legal = load_dataset("json", data_files="/kaggle/working/train_dataset_legal.json", split="train")
corpus_dataset_legal = concatenate_datasets([train_dataset_legal,val_dataset_legal,test_dataset_legal])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

## 4.Compute task retrieval of model-pretrained

In [14]:
 # Convert the datasets to dictionaries
corpus = dict(
    zip(corpus_dataset_legal["id"], corpus_dataset_legal["positive"])
)  # Our corpus (cid => document)
queries = dict(
    zip(test_dataset_legal["id"], test_dataset_legal["anchor"])
)  # Our queries (qid => question)
 
# Create a mapping of relevant document (1 in our case) for each query
relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for q_id in queries:
    relevant_docs[q_id] = [q_id]

In [15]:
matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=corpus,
        relevant_docs=relevant_docs,
        name=f"dim_{dim}",
        truncate_dim=dim,  # Truncate the embeddings to a certain dimension
        score_functions={"cosine": cos_sim},
    )
    matryoshka_evaluators.append(ir_evaluator)
# Create a sequential evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

### 5. Define loss function with Matryoshka Representation

In [16]:

model = SentenceTransformer(
    model_id,
    model_kwargs={"attn_implementation": "sdpa"},
    model_card_data=SentenceTransformerModelCardData(
        language="vi",
        license="apache-2.0",
        model_name="bkai-fine-tuned-legal",
    ),
)

In [17]:
matryoshka_dimensions = [768, 512, 256, 128, 64]  # Important: large to small
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

## 6.Fine-tune embedding model

In [18]:
 # Early stopping callback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=1 ,early_stopping_threshold=0.001)


In [ ]:
training_args=SentenceTransformerTrainingArguments(
        output_dir="bkai-fine-tuned-legal",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=52,
        per_device_eval_batch_size=52,
        gradient_accumulation_steps=12,
        num_train_epochs=8,
        disable_tqdm=False,
        lr_scheduler_type="cosine",
        batch_sampler=BatchSamplers.NO_DUPLICATES,
        weight_decay=0.2,
        save_total_limit=1,
        optim="adamw_torch_fused",
        fp16=True,
        max_grad_norm=0.65,
        warmup_ratio=0.15,
        group_by_length=True,
        report_to="wandb",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
from sentence_transformers import SentenceTransformerTrainer
 
trainer = SentenceTransformerTrainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset_legal.select_columns(
        ["positive", "anchor"]
    ),
    eval_dataset=val_dataset_legal.select_columns(["positive", "anchor"]),
    callbacks=[early_stopping_callback],
    loss=train_loss,
    evaluator=evaluator,
)

In [21]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()
 
# save the best model
trainer.save_model()
 

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.
wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250520_014440-wkv45cma
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run bkai-fine-tuned-legal
wandb: ⭐️ View project at https://wandb.ai/hdang1696-no-work-experience/sentence-transformers
wandb: 🚀 View run at https://wandb.ai/hdang1696-no-work-experience/sentence-transformers/runs/wkv45cma


Epoch,Training Loss,Validation Loss,Dim 768 Cosine Accuracy@1,Dim 768 Cosine Accuracy@3,Dim 768 Cosine Accuracy@5,Dim 768 Cosine Accuracy@10,Dim 768 Cosine Precision@1,Dim 768 Cosine Precision@3,Dim 768 Cosine Precision@5,Dim 768 Cosine Precision@10,Dim 768 Cosine Recall@1,Dim 768 Cosine Recall@3,Dim 768 Cosine Recall@5,Dim 768 Cosine Recall@10,Dim 768 Cosine Ndcg@10,Dim 768 Cosine Mrr@10,Dim 768 Cosine Map@100,Dim 512 Cosine Accuracy@1,Dim 512 Cosine Accuracy@3,Dim 512 Cosine Accuracy@5,Dim 512 Cosine Accuracy@10,Dim 512 Cosine Precision@1,Dim 512 Cosine Precision@3,Dim 512 Cosine Precision@5,Dim 512 Cosine Precision@10,Dim 512 Cosine Recall@1,Dim 512 Cosine Recall@3,Dim 512 Cosine Recall@5,Dim 512 Cosine Recall@10,Dim 512 Cosine Ndcg@10,Dim 512 Cosine Mrr@10,Dim 512 Cosine Map@100,Dim 256 Cosine Accuracy@1,Dim 256 Cosine Accuracy@3,Dim 256 Cosine Accuracy@5,Dim 256 Cosine Accuracy@10,Dim 256 Cosine Precision@1,Dim 256 Cosine Precision@3,Dim 256 Cosine Precision@5,Dim 256 Cosine Precision@10,Dim 256 Cosine Recall@1,Dim 256 Cosine Recall@3,Dim 256 Cosine Recall@5,Dim 256 Cosine Recall@10,Dim 256 Cosine Ndcg@10,Dim 256 Cosine Mrr@10,Dim 256 Cosine Map@100,Dim 128 Cosine Accuracy@1,Dim 128 Cosine Accuracy@3,Dim 128 Cosine Accuracy@5,Dim 128 Cosine Accuracy@10,Dim 128 Cosine Precision@1,Dim 128 Cosine Precision@3,Dim 128 Cosine Precision@5,Dim 128 Cosine Precision@10,Dim 128 Cosine Recall@1,Dim 128 Cosine Recall@3,Dim 128 Cosine Recall@5,Dim 128 Cosine Recall@10,Dim 128 Cosine Ndcg@10,Dim 128 Cosine Mrr@10,Dim 128 Cosine Map@100,Dim 64 Cosine Accuracy@1,Dim 64 Cosine Accuracy@3,Dim 64 Cosine Accuracy@5,Dim 64 Cosine Accuracy@10,Dim 64 Cosine Precision@1,Dim 64 Cosine Precision@3,Dim 64 Cosine Precision@5,Dim 64 Cosine Precision@10,Dim 64 Cosine Recall@1,Dim 64 Cosine Recall@3,Dim 64 Cosine Recall@5,Dim 64 Cosine Recall@10,Dim 64 Cosine Ndcg@10,Dim 64 Cosine Mrr@10,Dim 64 Cosine Map@100,Sequential Score
1,1.019400,0.353243,0.226886,0.431042,0.516525,0.683726,0.226886,0.143681,0.103305,0.068373,0.226886,0.431042,0.516525,0.683726,0.432697,0.355359,0.366100,0.231767,0.425743,0.515549,0.681495,0.231767,0.141914,0.103110,0.068149,0.231767,0.425743,0.515549,0.681495,0.433030,0.356691,0.367647,0.226328,0.426998,0.513318,0.681216,0.226328,0.142333,0.102664,0.068122,0.226328,0.426998,0.513318,0.681216,0.431213,0.354206,0.364911,0.221587,0.416818,0.504114,0.672291,0.221587,0.138939,0.100823,0.067229,0.221587,0.416818,0.504114,0.672291,0.422924,0.346341,0.357254,0.201785,0.398968,0.479849,0.643564,0.201785,0.132989,0.095970,0.064356,0.201785,0.398968,0.479849,0.643564,0.400399,0.325539,0.336916,0.400399
2,0.274600,0.186919,0.348766,0.485567,0.560312,0.719007,0.348766,0.161856,0.112062,0.071901,0.348766,0.485567,0.560312,0.719007,0.509353,0.445637,0.455293,0.346395,0.476084,0.550132,0.710361,0.346395,0.158695,0.110026,0.071036,0.346395,0.476084,0.550132,0.710361,0.503046,0.440204,0.450410,0.345977,0.476503,0.549156,0.710919,0.345977,0.158834,0.109831,0.071092,0.345977,0.476503,0.549156,0.710919,0.502778,0.439730,0.449856,0.337889,0.472319,0.550969,0.708548,0.337889,0.157440,0.110194,0.070855,0.337889,0.472319,0.550969,0.708548,0.498276,0.434433,0.444611,0.327012,0.453912,0.533817,0.692790,0.327012,0.151304,0.106763,0.069279,0.327012,0.453912,0.533817,0.692790,0.484067,0.420859,0.431632,0.484067
3,0.114100,0.158426,0.370520,0.482917,0.558918,0.720262,0.370520,0.160972,0.111784,0.072026,0.370520,0.482917,0.558918,0.720262,0.518114,0.457115,0.467070,0.364942,0.482917,0.557663,0.721378,0.364942,0.160972,0.111533,0.072138,0.364942,0.482917,0.557663,0.721378,0.515497,0.453383,0.463389,0.364942,0.482499,0.557802,0.717055,0.364942,0.160833,0.111560,0.071705,0.364942,0.482499,0.557802,0.717055,0.514352,0.453161,0.463353,0.364942,0.478315,0.554734,0.716636,0.364942,0.159438,0.110947,0.071664,0.364942,0.478315,0.554734,0.716636,0.513126,0.451788,0.461914,0.348905,0.463115,0.536466,0.702691,0.348905,0.154372,0.107293,0.070269,0.348905,0.463115,0.536466,

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]